In [1]:
import lancedb
import pandas as pd 
import numpy as np 
import os 
import re
import ast
from sentence_transformers import SentenceTransformer
# from lancedb_init import *

/Users/adithisatish/TUM/Academics/SoSe24/guided_research/.env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: attempted relative import with no known parent package

In [117]:
data_dir = "../../europeancities-wikivoyage-tripadvisor/data-sources/"
wikivoyage_docs_dir = data_dir + "wikivoyage/cleaned/"
wikivoyage_listings_dir = data_dir + "wikvoyage/listings/"
airbnb_dir = data_dir + "airbnb/"

uri = "../database/wikivoyage-docs-db"
db = lancedb.connect(uri)

In [85]:
def create_chunks(city, country, text):
    for i, line in enumerate(text):
        if line[0] == "\n":
            del text[i]

    index = 0 
    chunks = []
    pattern = re.compile("==")
    ignore = re.compile("===")
    section = 'Introduction'
    for i, line in enumerate(text):
        if pattern.search(line) and not ignore.search(line):
            chunk = ''.join(text[index:i])
            chunks.append({
                'city': city,
                'country': country,
                'section': section,
                'text': chunk, 
                'combined': f'city: {city}, country: {country}, section: {section}, text: {chunk}'
            })
            index=i+1
            section = re.sub(pattern, '', line).strip()

    df = pd.DataFrame(chunks)
    return df

    # print(chunks)

In [86]:
# file_name = "Ufa.txt"
# with open(wikivoyage_docs_dir + file_name) as file: 
#         text = file.readlines()
# chunk_df = create_chunks("Ufa", text)
# chunk_df.head()

In [ ]:
def read_docs():
    df = pd.DataFrame()
    cities = pd.read_csv("../city_abstracts_embeddings.csv")
    for file_name in os.listdir(wikivoyage_docs_dir):
        city = file_name.split(".")[0]
        country = cities[cities['city'] == city]['country'].item()
        with open(wikivoyage_docs_dir + file_name) as file: 
            text = file.readlines()
            chunk_df = create_chunks(city, country, text)
            df = pd.concat([df,chunk_df])

    return df

def preprocess_df(df):
    section_counts = df['section'].value_counts()
    sections_to_keep = section_counts[section_counts > 150].index
    filtered_df = df[df['section'].isin(sections_to_keep)]

    return filtered_df

def compute_wv_docs_embeddings(df):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    vector_dimension = model.get_sentence_embedding_dimension()
    
    print("Computing embeddings")
    embeddings = []
    for i, row in df.iterrows():
        emb = model.encode(row['combined'], show_progress_bar=True).tolist()
        embeddings.append(emb)

    print("Finished computing embeddings for wikivoyage documents.")
    df['vector'] = embeddings
    # df.to_csv(wv_embeddings + "wikivoyage-listings-embeddings.csv")
    # print("Finished saving file.")
    return df

# embeddings = compute_wv_docs_embeddings(filtered_df)

In [ ]:
df = read_docs()
filtered_df = preprocess_df(df)

In [87]:
# df = pd.DataFrame()
# cities = pd.read_csv("../city_abstracts_embeddings.csv")
# for file_name in os.listdir(wikivoyage_docs_dir):
#     city = file_name.split(".")[0]
#     country = cities[cities['city'] == city]['country'].item()
#     with open(wikivoyage_docs_dir + file_name) as file: 
#         text = file.readlines()
#         chunk_df = create_chunks(city, country, text)
#         df = pd.concat([df,chunk_df])

In [88]:
df.head()

,city,country,section,text,combined
0,Ufa,Russia,Introduction,'''[http://www.ufacity.info/ Ufa]{{Dead link|d...,"city: Ufa, country: Russia, section: Introduct..."
1,Ufa,Russia,Understand,\nUfa is one of the shortest place names in Ru...,"city: Ufa, country: Russia, section: Understan..."
2,Ufa,Russia,Get in,===By plane===\n* {{listing | type=go\n| name=...,"city: Ufa, country: Russia, section: Get in, t..."
3,Ufa,Russia,Get around,===By public transport===\n'''Marshrutkas''' a...,"city: Ufa, country: Russia, section: Get aroun..."
4,Ufa,Russia,See,\nWhat to see in Ufa is a tricky question. On ...,"city: Ufa, country: Russia, section: See, text..."


In [89]:
cities = set(df['city'].unique())

In [90]:
city_docs = pd.read_csv("../city_abstracts_embeddings.csv")
city_wv = set(city_docs['city'].unique())

In [91]:
city_wv - cities

{'Arad', 'Brest', 'Murcia', 'Penza', 'Valladolid'}

In [93]:
filtered_df

,city,country,section,text,combined
0,Ufa,Russia,Introduction,'''[http://www.ufacity.info/ Ufa]{{Dead link|d...,"city: Ufa, country: Russia, section: Introduct..."
1,Ufa,Russia,Understand,\nUfa is one of the shortest place names in Ru...,"city: Ufa, country: Russia, section: Understan..."
2,Ufa,Russia,Get in,===By plane===\n* {{listing | type=go\n| name=...,"city: Ufa, country: Russia, section: Get in, t..."
3,Ufa,Russia,Get around,===By public transport===\n'''Marshrutkas''' a...,"city: Ufa, country: Russia, section: Get aroun..."
4,Ufa,Russia,See,\nWhat to see in Ufa is a tricky question. On ...,"city: Ufa, country: Russia, section: See, text..."
...,...,...,...,...,...
5,Lille,France,Do,\n* {{do\n| name=Théatre Sebastopol | alt= | u...,"city: Lille, country: France, section: Do, tex..."
7,Lille,France,Buy,"*The open market, {{buy\n| name=Marché de Waze...","city: Lille, country: France, section: Buy, te..."
8,Lille,France,Eat,Food lovers will find themselves in a foodie h...,"city: Lille, country: France, section: Eat, te..."
9,Lille,France,Drink,"Lille has an amazing nightlife, wonderful bars...","city: Lille, country: France, section: Drink, ..."


In [94]:
# wv_embeddings = "../../europeancities-wikivoyage-tripadvisor/data-sources/wikivoyage/listings/"
# model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
# vector_dimension = model.get_sentence_embedding_dimension()

def compute_wv_docs_embeddings(df):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    vector_dimension = model.get_sentence_embedding_dimension()
    
    print("Computing embeddings")
    embeddings = []
    for i, row in df.iterrows():
        emb = model.encode(row['combined'], show_progress_bar=True).tolist()
        embeddings.append(emb)

    print("Finished computing embeddings for wikivoyage documents.")
    df['vector'] = embeddings
    # df.to_csv(wv_embeddings + "wikivoyage-listings-embeddings.csv")
    # print("Finished saving file.")
    return df

embeddings = compute_wv_docs_embeddings(filtered_df)

Computing embeddings


Batches: 100%|██████████| 1/1 [00:00<00:00, 44.08it/s]

Finished computing embeddings for wikivoyage documents.



/var/folders/07/81pmlyln5szbq49h1zdqjq1h0000gn/T/ipykernel_12115/2938155819.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['embeddings'] = embeddings


In [95]:
embeddings

,city,country,section,text,combined,embeddings
0,Ufa,Russia,Introduction,'''[http://www.ufacity.info/ Ufa]{{Dead link|d...,"city: Ufa, country: Russia, section: Introduct...","[0.09404498338699341, -0.0212897676974535, -0...."
1,Ufa,Russia,Understand,\nUfa is one of the shortest place names in Ru...,"city: Ufa, country: Russia, section: Understan...","[0.03183458000421524, 0.023557119071483612, -0..."
2,Ufa,Russia,Get in,===By plane===\n* {{listing | type=go\n| name=...,"city: Ufa, country: Russia, section: Get in, t...","[0.08447606861591339, -0.02755945362150669, -0..."
3,Ufa,Russia,Get around,===By public transport===\n'''Marshrutkas''' a...,"city: Ufa, country: Russia, section: Get aroun...","[0.0026172951329499483, -0.0533800944685936, 0..."
4,Ufa,Russia,See,\nWhat to see in Ufa is a tricky question. On ...,"city: Ufa, country: Russia, section: See, text...","[0.0736846774816513, 0.07103532552719116, -0.1..."
...,...,...,...,...,...,...
5,Lille,France,Do,\n* {{do\n| name=Théatre Sebastopol | alt= | u...,"city: Lille, country: France, section: Do, tex...","[0.007476135157048702, -0.07160720229148865, -..."
7,Lille,France,Buy,"*The open market, {{buy\n| name=Marché de Waze...","city: Lille, country: France, section: Buy, te...","[0.010724762454628944, 0.01523562241345644, 0...."
8,Lille,France,Eat,Food lovers will find themselves in a foodie h...,"city: Lille, country: France, section: Eat, te...","[-0.028721176087856293, -0.019028617069125175,..."
9,Lille,France,Drink,"Lille has an amazing nightlife, wonderful bars...","city: Lille, country: France, section: Drink, ...","[0.03291141241788864, -0.028743581846356392, 0..."


In [97]:
embeddings.rename(columns={'embeddings':'vector'}, inplace=True)
# embeddings['vector'] = embeddings['vector'].apply(ast.literal_eval)

/var/folders/07/81pmlyln5szbq49h1zdqjq1h0000gn/T/ipykernel_12115/3587157459.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  embeddings.rename(columns={'embeddings':'vector'}, inplace=True)


ValueError: malformed node or string: [0.09404498338699341, -0.0212897676974535, -0.04158734902739525, 0.06594733148813248, 0.014034571126103401, -0.022166749462485313, 0.01671741157770157, -0.02555234357714653, -0.085683174431324, 0.012842592783272266, -0.01948041282594204, -0.05766274407505989, -0.05075592175126076, 0.0807848870754242, -0.02325591631233692, -0.011258046142756939, 0.008875174447894096, -0.05967216566205025, 0.01106437761336565, -0.04447950795292854, -0.038516607135534286, -0.07270991802215576, 0.11787271499633789, 0.008689041249454021, 0.005068542435765266, 0.014361604116857052, 0.03075834922492504, -0.0009561151382513344, 0.013904988765716553, -0.05798893794417381, 0.020920459181070328, 0.09383182227611542, 0.05381021276116371, -0.033570174127817154, 0.004245661664754152, 0.06736408174037933, 0.05025274679064751, -0.03945622220635414, -0.032128673046827316, 0.046130843460559845, 0.027982642874121666, -0.07779470086097717, 0.038396432995796204, 0.06420920789241791, 0.

In [99]:
db.drop_table("wikivoyage_documents", ignore_missing=True)
db.create_table("wikivoyage_documents", data=embeddings)

LanceTable(connection=LanceDBConnection(/Users/adithisatish/TUM/Academics/SoSe24/guided_research/rag-sustainable-trs/vectordb/../database/wikivoyage-docs-db), name="wikivoyage_documents")

In [100]:
def embed_query(query):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    # vector_dimension = model.get_sentence_embedding_dimension()   
    embedding = model.encode(query).tolist()
    return embedding

query = "Cities in Europe with a great nightlife and art museums"
query_embedding = embed_query(query)
table = db.open_table("wikivoyage_documents")
results = table.search(query_embedding).limit(5).to_list()

In [102]:
city_lists = [f"city: {r['city']}, country: {r['country']}, section: {r['section']}, text: {r['text']}" for r in results]

In [105]:
print("The recommended cities are:")
for city in results:
    print(f"{city['city']}, {city['country']}")

The recommended cities are:
Varna, Bulgaria
Braga, Portugal
Amsterdam, Netherlands
Burgas, Bulgaria
Lyon, France


In [109]:
path_embeddings = data_dir + "wikivoyage/embeddings/"
embeddings.to_csv(path_embeddings + "chunked-embeddings.csv")

In [112]:
len(embeddings)

1544

In [121]:
emb_cities = set(embeddings['city'].unique())

In [120]:
listings = pd.read_csv("../../europeancities-wikivoyage-tripadvisor/data-sources/wikivoyage/listings/wikivoyage-listings-cleaned.csv")
listing_cities = set(listings['city'].unique())

In [122]:
emb_cities ^ listing_cities

{'Batman',
 'Erzincan',
 'Isparta',
 'Istanbul',
 'Kahramanmaras',
 'Luxembourg',
 'Mykolaiv',
 'Nalchik',
 'Orenburg',
 'Saratov',
 'Siirt',
 'Stavropol',
 'Vinnytsia',
 'Zaporizhzhia'}